In [27]:
# 필요 모듈 Import

import cv2
import numpy as np
from skimage import io
from skimage.transform import resize
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from gtts import gTTS
import hgtk
from io import BytesIO
import webbrowser

In [23]:
# 동화 및 아이한글 크롤링 자료 중 3글자인 것들만 추출하여 저장한 파일 불러오기. (사전 전처리 및 저장 진행)

number3 = open('C:/Users/user/Desktop/PPT/data/number3/3str.txt' ,encoding='utf-8').readlines()

In [24]:
n3 = [] # '\n' 기호 제거 (저장하면 나타나지는 않지만 깔끔한 정돈을 위하여.)

for line in number3:
    replace_line = line.replace('\n','')
    n3.append(replace_line)

In [25]:
# 중복 제거 - 기존 파일 생성할 때 했었어야 하는 부분인데 빠뜨려서 추가 진행.

new_list = []
for v in n3:
    if v not in new_list:
        new_list.append(v)
print(len(new_list))

423


In [5]:
# hgtk 자-모음 분리 모듈

dc= []
for i in range(0, len(new_list)):
    dc.append(hgtk.text.decompose(new_list[i]))

In [9]:
# 'ㅂ'/'ㄷ' 된소리되기 문제 생성, 이런 식으로 조건을 걸어서 문제를 생성해낼 수 있음. ex)유음화, 비음화, 된소리되기, 특정 단어 등등...

select2="ㅂᴥㄷ"
hard_list= []
for select in dc:
    if select2 in select or "ㄷᴥㅂ" in select :
        hard_list.append(select)

In [7]:
# hgtk 모듈의 경우, 분리 간에 'ᴥ' 기호가 추가가 되는데 이는 필요시 정규식을 통해 제거가 가능.

print(hard_list)

['ㅈㅡㄹᴥㄱㅓㅂᴥㄷㅏᴥ', 'ㅊㅏᴥㄱㅏㅂᴥㄷㅏᴥ', 'ㄴㅗㄹᴥㄹㅏㅂᴥㄷㅏᴥ', 'ㅅㅏᴥㄴㅏㅂᴥㄷㅏᴥ', 'ㅁㅜᴥㅅㅓㅂᴥㄷㅏᴥ', 'ㄱㅗㄷᴥㅂㅏᴥㄹㅗᴥ']


In [11]:
# 비음, 유음화가 진행된 단어 추출 (compose는 decompose 반대, 다시 합치는 명령어.)

cm = []
for i in range(0, len(hard_list)):
    cm.append(hgtk.text.compose(hard_list[i]))
print(cm)

['즐겁다', '차갑다', '놀랍다', '사납다', '무섭다', '곧바로']


In [12]:
# 위의 단어들 가운데 임의로 1개 추출.

import random

index = random.randint(0,len(cm))
print(cm[index])

무섭다


In [14]:
# 받아쓰기 정답과 비교를 위한 과정 (아래 코드에서 사용)

model_text = cm[index]

In [15]:
# 문제를 읽어주는 파일 생성 및 웹 재생.

tts = gTTS(text=cm[index], lang='ko')
tts.save("bskyvision.mp3")

In [16]:
webbrowser.open('bskyvision.mp3')

True

In [21]:
# 한글 데이터 모델 불러오기

model = keras.models.load_model('hand_written_korean_classification.hdf5') # 총 46,060개 한글 손 글씨 이미지를 바탕으로 만들어진 모델
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [18]:
# 한글 데이터셋에 포함되어 있는 라벨 불러오기

label = pd.read_table('./label.txt',header=None)
label = label.reset_index(0)

In [19]:
drawing = False
ix, iy = -1, -1

In [26]:
def draw_circle(event, x, y, flags, param):
     global ix, iy, drawing, mode

     if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

     if event == cv2.EVENT_MOUSEMOVE:
          if drawing == True:
                cv2.circle(img, (x, y), 5, (255, 255, 255), -1)

     elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.circle(img, (x, y), 5, (255, 255, 255), -1)

def Answer(img):
    img = cv2.resize(img, dsize=(96, 32), interpolation=cv2.INTER_AREA)
    
    word1 = img[:,:32,:]
    word2 = img[:,32:64,:]
    word3 = img[:,64:,:]
    
    word1 = cv2.resize(word1, dsize=(32,32))
    word1 = 255 - word1
    word1 = word1.reshape(1,32,32,3)
    word1 = word1/255
    
    word2 = cv2.resize(word2, dsize=(32,32))
    word2 = 255 - word2
    word2 = word2.reshape(1,32,32,3)
    word2 = word2/255
    
    word3 = cv2.resize(word3, dsize=(32,32))
    word3 = 255 - word3
    word3 = word3.reshape(1,32,32,3)
    word3 = word3/255
   
   # img = img.reshape(32, 32, 3)
   # img = 255 - img
   # img = cv2.resize(img, dsize=(32, 32))

    
   # img = img.reshape(1,32,32,3)
   # img = img/255
   
    print('입력값 : ', label.iloc[int(model.predict_classes(word1)),1])
    a1 = label.iloc[int(model.predict_classes(word1)),1]
    print('입력값 : ', label.iloc[int(model.predict_classes(word2)),1])
    a2 = label.iloc[int(model.predict_classes(word2)),1]
    print('입력값 : ', label.iloc[int(model.predict_classes(word3)),1])
    a3 = label.iloc[int(model.predict_classes(word3)),1]
    answer = a1+a2+a3
    print('사용자답안 :', answer)
    if(model_text == answer):
        path = 'C:/Users/user/Desktop/PPT/answer.png'
        image = cv2.imread(path)
        cv2.imshow('Good',image)
    else :
        path = 'C:/Users/user/Desktop/PPT/fail.png'
        image1 = cv2.imread(path)
        cv2.imshow('No',image1)
    
img = np.zeros((300, 900, 3), np.uint8)
img[:,300,:]=255
img[:,600,:]=255
cv2.namedWindow('Gakduki')
cv2.setMouseCallback('Gakduki', draw_circle)


while(1):
     cv2.imshow('Gakduki', img)
     k = cv2.waitKey(1) & 0xFF

     if k == 27: # ESC-CLOSE
        break
     elif k == 13: 
          Answer(img)
          img = np.zeros((300, 900, 3), np.uint8)
          img[:,300,:]=255
          img[:,600,:]=255

#---------------------------------------
cv2.destroyAllWindows()

입력값 :  꾸
입력값 :  극
입력값 :  극
사용자답안 : 꾸극극
